## Bing news

### API Method

> METODO REQUEST

> https://learn.microsoft.com/en-us/bing/search-apis/bing-news-search/reference/query-parameters

> https://www.microsoft.com/en-us/bing/apis/pricing

In [20]:
import os
os.chdir('C:\\Users\\Vinim\\Documents\\github\\tcc_vinicius')

In [21]:
import deployed_azure.bing_search as bing
import pandas as pd
import deployed_azure.util as util
from datetime import datetime 

today = datetime.today().strftime('%Y-%m-%d')

In [22]:
def process_search(searches, result_count, period="day"):
    dfs = []
    today = datetime.today().strftime('%Y-%m-%d-%Hh-%Mmin')
    for search_term in searches:
        result = bing.search_news(search_term, result_count, period)
        result['full_text'] = result['url'].apply(util.extract_url_content)
        result['key'] = search_term
        result['data_de_busca'] = today
        dfs.append(result)
        
    dfs_result = pd.concat(dfs) 
    return dfs_result

In [23]:
searches = [
    "PETR4", "Petrobras", "PETROLEO BRASILEIRO S A PETROBRAS",
    "VALE3", "Vale", "VALE S.A.",
    "ITUB4", "Itaú Unibanco", "ITAU UNIBANCO HOLDING S.A.",
    "ABEV3", "Ambev", "AMBEV S.A.",
    "WEGE3", "WEG", "WEG SA",
    "BBDC4", "Banco Bradesco", "BANCO BRADESCO S.A.",
    "BBAS3", "Banco do Brasil", "BANCO DO BRASIL SA",
    "BPAC11", "Banco BTG Pactual", "BANCO BTG PACTUAL S.A.",
    "SANB3", "Banco Santander", "BANCO SANTANDER (BRASIL) S.A.",
    "ITSA4", "Itaúsa", "ITAUSA S.A."
    ]
df_result = process_search(searches,10)

In [24]:
# name = 'data//result_2024-04-10-11_2.parquet'


In [25]:
name = f'data//result_{today}.parquet'
df_result.to_parquet(name)


--------------

# blob storage 

In [26]:
import os, uuid
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [27]:
def blob_send_itens(local_file_name):
    try:
        print("Azure Blob Storage Python")

        account_url = "https://appstocknewsprediction.blob.core.windows.net"
        default_credential = DefaultAzureCredential()

        # Create the BlobServiceClient object
        blob_service_client = BlobServiceClient(account_url, credential=default_credential)

        # Create a file in the local data directory to upload and download
        container_name = 'result-function-stocknews'
        local_path = 'data'
        # local_file_name = 'result_2024-01-13.parquet'
        upload_file_path = os.path.join(local_path, local_file_name)

        # Create a blob client using the local file name as the name for the blob
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=local_file_name)

        print("\nUploading to Azure Storage as blob:\n\t" + local_file_name)

        # Upload the created file
        with open(file=upload_file_path, mode="rb") as data:
            blob_client.upload_blob(data)
        return f'done:{local_file_name}'
    
    except Exception as ex:    
        result =  f'Exception:{ex}' 
        print(result)
        return result 

In [28]:
item_sent = blob_send_itens(name[6:])

Azure Blob Storage Python

Uploading to Azure Storage as blob:
	result_2024-04-10-11_2.parquet
